In [261]:
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from tqdm import tqdm

In [262]:
iris_set = load_iris()

[dataset specification](https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.load_iris.html)
## IRIS Dataset
**class:** [setosa, versicolour, virginica]
**Number of Instances**: `150` (50 in each of three classes)
**Number of Attributes**: `4` numeric, predictive attributes and the class
**Attribute Information**: `sepal length`, `sepal width`, `petal length`, `petal width`

# 선형 회귀분석
선형 회귀분석(Linear Regression)란 하나 이상의 설명변수와 그에 대한 스칼라 반응(sclar response)을 통한 모델링입니다. 그 중에서도 선형적인 접근을 하는 것을 지칭합니다.
* 자료로 예측된 매개변수, 그로 만들어진 선형 예측자 함수(linear predictor function)로 입출력 관계를 모사합니다—이를 선형모델(linear model)이라 합니다. 대부분의 경우에 매개변수를 구하기 위한 알고리즘을 위해 설명변수와 모델 간의 반응과 그 조건부 평균은 결국 아핀 함수로 나타낼 수 있어야 합니다.
* 일반적으로 최소 제곱법(least square)이 적합을 위해 사용됩니다. 이때, 적합이 제대로 작동하지 않은 상태를 적합성 결여(lack of fitting, LOF)이라 합니다—이를 피하기 위해 최소 제곱법에서 파생된 다양한 접근이 존재합니다.
* 선형회귀는 결합부 확률 분포(joint probability distribution)보다는 조건부 확률 분포(conditional probability distribution)에 초점을 가지고 있습니다.

In [263]:
class Dataset:
  def __init__(self, dataset, indices, transform=None, encoder=None):
    self.dataset, self.indices = dataset, indices
    self.transform, self.encoder = transform, encoder
  def __getitem__(self, item: int):
    idx = self.indices[item]
    feature, label = self.dataset.data[idx], self.dataset.target[idx]
    if self.transform: feature = self.transform(feature)
    if self.encoder: label = self.encoder(label)
    return feature, label
  def __len__(self): return len(self.indices)

## 손실함수 정의하기
손실함수(loss function)는 기계학습에서 모델의 예측값과 실제값 사이의 차이를 측정하는 함수입니다. 손실 함수는 모델이 얼마나 잘 수행되고 있는지를 수치적으로 나타내며, 이 값을 최소화하는 것이 모델 학습의 목표입니다. 개별적인 관찰에 대한 손실은 비용함수(cost function)라고 별칭합니다.
* $\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$
* $\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$

In [264]:
# define loss functions(MSE, MAE)
def mean_squared_error(independent, dependent, weight):
  probability = np.dot(independent, weight)
  return np.mean((probability - dependent) ** 2)
# mean_squared_error

def mean_absolute_error(independent, dependent, weight):
  probability = np.dot(independent, weight)
  return np.mean(abs(probability - dependent))
# mean_absolute_error

## 수식화
$\left\{ y_i, x_{i1}, …, x_{ip} \right\}^n_{i=1}$를 통해 선형모델은 독립변수 y와 종속변수 $\beta$를 예측해야 합니다.
$$y_{i} = \beta_{0} + \beta_{1}x_{i1} + ... + \beta_{p}x_{ip} + \epsilon_{i} = \beta_{p}x_{ip} + \epsilon_{i}$$
* $y$는 관측값의 벡터입니다.
* $x$는 열 벡터 $x_i$의 행렬 또는 다차원의 행 벡터 $x_j$입니다.
* $\beta$는 $p+1$ 차원의 매개변수 벡터입니다.
* $\epsilon$는 $\epsilon_i$의 벡터입니다.

In [265]:
# define LinearRegression
class LinearRegression:
  def __init__(self, n_inpt): self.weight = np.zeros(shape=(n_inpt))
  def gdr(self, x, y, lr):
    indications = self.forward(x)
    self.weight -= (lr / x.shape[0]) * np.dot(x.T, (indications - y))
  # gdr
  def train(self, dataset, iters: int, lr=0.01):
    for _ in range(iters):
      for feature, label in dataset: self.gdr(feature, label, lr=lr)
  # train
  def forward(self, x): return np.dot(x, self.weight)
# LogisticRegression

## 경사 하강법
경사하강법(Gradient Descent)은 머신러닝과 최적화 분야에서 널리 사용되는 알고리즘으로, 함수의 최솟값을 찾기 위해 반복적으로 파라미터를 업데이트하는 방법입니다. 주로 모델의 손실을 최소화하는 데 사용되며, 이 과정에서 모델의 가중치를 조정합니다.
1. 가중치 $\theta$를 임의의 값을 초기화합니다.
2. 현재의 가중치 $\theta$에서 손실함수의 경사 $\nabla{f(\theta)}$를 계산합니다.
3. 가중치 갱신하기: $\theta_{\text{new}} \leftarrow \theta - \mathcal{n}\cdot\nabla{f(\theta)}$
4. 경사가 충분히 작아질 때까지 2 ~ 3을 반복합니다.

In [266]:
def GDR(model, lr):
  def _GDR(x, y):
    pred = model.forward(x)
    model.weight -= lr * np.dot(x.T, (pred - y))
  return _GDR

In [267]:
progress_bar = tqdm(range(10))

# init and train a model
model = LinearRegression(4)
optimizer = GDR(model, 0.001)
for _ in progress_bar:
  loss = 0.
  for feature, label in support_set:
    optimizer(feature, label)
    loss += mean_squared_error(feature, label, model.weight)
  progress_bar.set_postfix(loss=loss/len(support_set))

100%|██████████| 10/10 [00:00<00:00, 480.38it/s, loss=0.0912]


## 검증하기
아래 검증 결과를 통해 해당 구현이 `0.92`의 정확도를 달성했음을 알 수 있습니다.

In [268]:
count, n_samples = 0, len(query_set)
for feature, label in support_set:
  pred = model.forward(feature)
  if round(pred) == label: count += 1
print(f"accuracy: {count / n_samples:.2f}({count}/{n_samples})")

accuracy: 0.92(46/50)
